In [1]:
import numpy as np
import pandas as pd

from helpers import cleaner as cl
from helpers import common as cm
from helpers import trainer as tr

In [2]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import make_scorer, mean_squared_error, precision_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures

In [3]:
exchange = 'Bittrex'

In [4]:
df_shared = cm.load_data('data/shared/{}/transformed.csv'.format(exchange))
df_original = cm.load_data('data/shared/{}/original.csv'.format(exchange))

# Shared
---

In [5]:
# Variables
currency_to_predict = 'LTC'
all_columns = df_shared.columns
label_column_name = 'price_close'
column_name = '{}_{}'.format(label_column_name, currency_to_predict)

In [6]:
currencies = cm.all_currencies()
all_dates = sorted(list(set(df_original['date'])))
all_hours = cm.all_hours()
all_minutes = cm.all_minutes()

In [7]:
labels_original = pd.DataFrame(columns=['label'])
labels_original['label'] = df_original[[column_name]]

# Logistic Regression
---

In [8]:
df_cleaned, labels_cleaned = cl.clean_data_using_labels(
    df_shared,
    labels_original,
    1,
    lambda x, y: x > y,
)

In [9]:
X_train, y_train, X_test, y_test = tr.training_and_test_sets(
    df_cleaned,
    labels_cleaned,
    ['date', 'hour', 'minute'],
)

In [10]:
param_grid = {
    'C': np.logspace(-2, 10, num=3),
    'max_iter': [1000, 2000, 3000],
    'random_state': [0],
    'solver': [
        'newton-cg',
        'lbfgs',
        'liblinear',
        'sag',
    ],
}

gs = GridSearchCV(
    LogisticRegression(),
    param_grid,
    cv=3,
    scoring=make_scorer(precision_score),
    verbose=10,
)

In [11]:
training_opts = {
    'features_to_use': X_train.columns[2:],
    'n_times_to_try': 1,
    'coef_extractor': lambda x: x.coef_[0],
    'grid_search': gs,
    'data': X_train,
    'labels': y_train,
    'columns_to_stratify': None,
}

score, model, feature_importances, features_to_use = tr.train_using_best_features(training_opts)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV] C=0.01, max_iter=1000, random_state=0, solver=newton-cg .........
[CV]  C=0.01, max_iter=1000, random_state=0, solver=newton-cg, score=0.546006, total=   1.5s
[CV] C=0.01, max_iter=1000, random_state=0, solver=newton-cg .........


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s


[CV]  C=0.01, max_iter=1000, random_state=0, solver=newton-cg, score=0.510040, total=   1.5s
[CV] C=0.01, max_iter=1000, random_state=0, solver=newton-cg .........


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.0s remaining:    0.0s


[CV]  C=0.01, max_iter=1000, random_state=0, solver=newton-cg, score=0.493540, total=   1.8s
[CV] C=0.01, max_iter=1000, random_state=0, solver=lbfgs .............


...

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[CV]  C=10000000000.0, max_iter=2000, random_state=0, solver=sag, score=0.582642, total= 1.9min
[CV] C=10000000000.0, max_iter=2000, random_state=0, solver=sag ......


/usr/local/lib/python3.6/site-packages/sklearn/linear_model/sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[CV]  C=10000000000.0, max_iter=2000, random_state=0, solver=sag, score=0.580628, total= 1.8min
[CV] C=10000000000.0, max_iter=2000, random_state=0, solver=sag ......
[CV]  C=10000000000.0, max_iter=2000, random_state=0, solver=sag, score=0.565011, total= 1.7min
[CV] C=10000000000.0, max_iter=3000, random_state=0, solver=newton-cg 
[CV]  C=10000000000.0, max_iter=3000, random_state=0, solver=newton-cg, score=0.577458, total=   7.1s
[CV] C=10000000000.0, max_iter=3000, random_state=0, solver=newton-cg 
[CV]  C=10000000000.0, max_iter=3000, random_state=0, solver=newton-cg, score=0.581384, total=   7.5s
[CV] C=10000000000.0, max_iter=3000, random_state=0, solver=newton-cg 
[CV]  C=10000000000.0, max_iter=3000, random_state=0, solver=newton-cg, score=0.556526, total=   6.8s
[CV] C=10000000000.0, max_iter=3000, random_state=0, solver=lbfgs ....
[CV]  C=10000000000.0, max_iter=3000, random_state=0, solver=lbfgs, score=0.578053, total=   8.4s
[CV] C=10000000000.0, max_iter=3000, random_state

[Parallel(n_jobs=1)]: Done 108 out of 108 | elapsed: 41.6min finished
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [13]:
# y_predict = model.predict(X_test)
# precision_score(y_test, y_predict)
score

0.60855908912446011

# Linear Regression
---

In [ ]:
df_cleaned, labels_cleaned = cl.clean_data_using_labels(
    df_shared,
    labels_original,
    1,
    lambda x, y: (x - y) / y
#     lambda x, y: x
)

In [ ]:
X_train, y_train, X_test, y_test = tr.training_and_test_sets(
    df_cleaned,
    labels_cleaned,
    ['date', 'hour', 'minute'],
)

In [ ]:
r_model = LinearRegression()

param_grid = {}

gs = GridSearchCV(
    r_model,
    param_grid,
    cv=3,
    scoring=make_scorer(mean_squared_error),
    verbose=0,
)

In [ ]:
gs.fit(X_train, cm.labels_to_array(y_train))

In [ ]:
best_model = gs.best_estimator_
cv_results = gs.cv_results_

feature_importances = sorted(
    zip(
        best_model.coef_,
        X_train.columns,
    ),
    key=lambda x: np.absolute(x[0]),
    reverse=True,
)
best_features = [f for s, f in feature_importances]
feature_importances[0:20]

In [ ]:
y_predict = best_model.predict(X_test)
mean_squared_error(y_test, y_predict)